In [2]:
import pennylane as qml
from pennylane import numpy as np
import torch
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, f1_score
import time

In [ ]:
n_qubits = 2
noise_prob = 0.3   # Change this to set noise probability (0.0 = noiseless)
noise_type = "AmplitudeDamping"  # Options: "Depolarizing", "AmplitudeDamping", "PhaseDamping", "BitFlip", "PhaseFlip"

In [4]:
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Binary classification (Setosa vs Versicolor)
mask = y < 2
X = X[mask]
y = y[mask]

# Standardize
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Use only first 2 features
X = X[:, :2]
y = y.astype(float)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Torch tensors in float64 (PennyLane default)
X_train = torch.tensor(X_train, dtype=torch.float64)
y_train = torch.tensor(y_train, dtype=torch.float64)
X_test = torch.tensor(X_test, dtype=torch.float64)
y_test = torch.tensor(y_test, dtype=torch.float64)

In [5]:
dev = qml.device("default.mixed", wires=n_qubits)  # supports noise

def feature_encoding(x):
    for i in range(n_qubits):
        qml.RX(x[i], wires=i)

def shallow_HEA(weights):
    for i in range(n_qubits):
        qml.RX(weights[i, 0], wires=i)
        qml.RZ(weights[i, 1], wires=i)
    qml.CNOT(wires=[0, 1])

def apply_noise(noise_type, prob):
    """Apply chosen noise model to all qubits."""
    for i in range(n_qubits):
        if noise_type == "Depolarizing":
            qml.DepolarizingChannel(prob, wires=i)
        elif noise_type == "AmplitudeDamping":
            qml.AmplitudeDamping(prob, wires=i)
        elif noise_type == "PhaseDamping":
            qml.PhaseDamping(prob, wires=i)
        elif noise_type == "BitFlip":
            qml.BitFlip(prob, wires=i)
        elif noise_type == "PhaseFlip":
            qml.PhaseFlip(prob, wires=i)

@qml.qnode(dev, interface="torch")
def circuit(weights, x):
    feature_encoding(x)
    apply_noise(noise_type, noise_prob)   # Noise after encoding
    shallow_HEA(weights)
    apply_noise(noise_type, noise_prob)   # Noise after ansatz
    return qml.expval(qml.PauliZ(0))


In [6]:
class QuantumClassifier(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = torch.nn.Parameter(0.01 * torch.randn(n_qubits, 2, dtype=torch.float64))

    def forward(self, x):
        outputs = []
        for xi in x:
            outputs.append(circuit(self.weights, xi))
        return torch.stack(outputs).reshape(-1, 1) * 0.5 + 0.5


In [7]:
model = QuantumClassifier()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.BCELoss()

epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(X_train)
    loss = loss_fn(y_pred, y_train.reshape(-1, 1))
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 5 == 0:
        acc = ((y_pred.detach().numpy() > 0.5) == y_train.numpy().reshape(-1, 1)).mean()
        print(f"Epoch {epoch+1:2d} | Loss: {loss.item():.4f} | Train Acc: {acc*100:.2f}%")


Epoch  5 | Loss: 1.0598 | Train Acc: 52.50%
Epoch 10 | Loss: 0.6722 | Train Acc: 57.50%
Epoch 15 | Loss: 0.5123 | Train Acc: 67.50%
Epoch 20 | Loss: 0.4571 | Train Acc: 82.50%
Epoch 25 | Loss: 0.4476 | Train Acc: 85.00%
Epoch 30 | Loss: 0.4524 | Train Acc: 86.25%


In [10]:
with torch.no_grad():
    start_time = time.perf_counter()

    preds = model(X_test).detach().numpy()  # Detach once
    preds_binary = (preds > 0.5).astype(int)

    end_time = time.perf_counter()
    inference_time_per_sample = (end_time - start_time) / len(X_test)

    y_true_np = y_test.detach().numpy().flatten()

    mse = mean_squared_error(y_true_np, preds.flatten())
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_true_np, preds.flatten())

    # F1 score
    y_true_int = y_test.detach().numpy().astype(int)
    f1 = f1_score(y_true_int, preds_binary)

print(f"Noise: {noise_type}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Inference time per sample: {inference_time_per_sample:.6f} sec")
print(f"No. of qubits: {n_qubits}")


Noise: AmplitudeDamping
MSE: 0.1526
RMSE: 0.3907
MAE: 0.3672
F1-score: 0.8889
Inference time per sample: 0.005239 sec
No. of qubits: 2
